# HuBMAP Release 1
Thomas Hopkins

## Overview

Here is the first attempt at training a UNet for the HuBMAP competition.

The code for this notebook is ported from my GitHub repository here: https://github.com/thomashopkins32/HuBMAP

In [ ]:
import os
import json

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import measure
from skimage.draw import polygon
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

In [ ]:
#%load_ext tensorboard.notebook
#%tensorboard --logdir runs

## Utilities

Here are some useful utilities that help in debugging, model training, and model evalutation.

I wrote a custom version of mAP for fun so use at your own risk.

In [ ]:
def accuracy(logits, labels):
    preds = torch.argmax(logits, dim=1)
    return torch.count_nonzero(preds == labels) / preds.shape[0]


def memory_usage_stats(model, optimizer, batch_size=1, device='cuda'):
    print(f'Starting memory: {torch.cuda.memory_allocated(device) * 1e-6}')
    model.to(device)
    print(f'After model sent to {device}: {torch.cuda.memory_allocated(device) * 1e-6}')
    for i in range(3):
        sample = torch.randn((batch_size, 3, 512, 512))
        print(f'Step {i}')
        before = torch.cuda.memory_allocated(device) * 1e-6
        out = model(sample.to(device)).sum()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After forward pass: {after}')
        print(f'Memory used by forward pass: {after - before}')
        out.backward()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After backward pass: {after}')
        optimizer.step()
        print(f'After optimizer step: {torch.cuda.memory_allocated(device) * 1e-6}')
    torch.cuda.empty_cache()


def memory_usage_stats_grad_scaler(model, optimizer, batch_size=1, device='cuda'):
    if device != 'cuda':
        print('This function requires device to be "cuda".')
        return
    print(f'Starting memory: {torch.cuda.memory_allocated(device) * 1e-6}')
    model.to(device)
    print(f'After model sent to {device}: {torch.cuda.memory_allocated(device) * 1e-6}')
    scaler = torch.cuda.amp.GradScaler()
    for i in range(3):
        sample = torch.randn((batch_size, 3, 512, 512))
        print(f'Step {i}')
        before = torch.cuda.memory_allocated(device) * 1e-6
        with torch.cuda.amp.autocast(dtype=torch.float16):
            out = model(sample.to(device)).sum()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After forward pass: {after}')
        print(f'Memory used by forward pass: {after - before}')
        scaler.scale(out).backward()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After backward pass: {after}')
        scaler.step(optimizer)
        print(f'After optimizer step: {torch.cuda.memory_allocated(device) * 1e-6}')
        scaler.update()
    torch.cuda.empty_cache()


def average_precision(prediction, target, iou_threshold=0.6):
    '''
    Computes the average precision (AP) for an instance segmentation task on a single image.

    Parameters
    ----------
    prediction : np.array
        Boolean prediction mask
    target : np.array
        Boolean ground-truth mask
    iou_threshold : float, optional
        Threshold at which to count predictions as positive predictions

    Returns
    -------
    AP : float
        Average precision score
    '''
    pred_regions, pred_region_count = measure.label(prediction, return_num=True)
    target_regions, target_region_count = measure.label(target, return_num=True)
    true_positives = 0
    false_positives = 0
    for p in range(1, pred_region_count + 1):
        pred_region_mask = pred_regions == p
        max_iou = 0.0
        for t in range(1, target_region_count + 1):
            target_region_mask = target_regions == t
            # Compute IoU this region pair
            intersection = np.logical_and(pred_region_mask, target_region_mask)
            union = np.logical_or(pred_region_mask, target_region_mask)
            intersection_count = np.count_nonzero(intersection)
            union_count = np.count_nonzero(union)
            if intersection_count > 0 and union_count > 0:
                iou = intersection_count / union_count
                max_iou = max(max_iou, iou)
        if max_iou > iou_threshold:
            true_positives += 1
        else:
            false_positives += 1
    if (true_positives == 0 and false_positives == 0) or target_region_count == 0:
        return 0.0
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / target_region_count

    return precision * recall


def mAP(predictions, targets, iou_threshold=0.6):
    averages = 0.0
    predictions = predictions.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    for p, t in zip(predictions, targets):
        averages += average_precision(p, t, iou_threshold=iou_threshold)
    return averages / len(predictions)


def logits_to_mask(logits):
    # TODO: update with only blood vessels
    return torch.argmax(torch.softmax(logits, dim=1), dim=1).type(torch.long)


def train_one_epoch(epoch, model, train_loader, loss_func, optimizer, writer=None, device='cpu', **kwargs):
    model.train()
    for d in train_loader:
        optimizer.zero_grad()
        x = d['image']
        y = d['mask']
        x = x.to(device)
        y = y.long().to(device)
        logits = model(x)
        loss = loss_func(logits, y)
        loss.backward()
        optimizer.step()
        if writer:
            with torch.no_grad():
                predictions = logits_to_mask(logits)
                mAP_value = mAP(predictions, y, **kwargs)
                writer.add_scalar("Loss/train", loss.item(), global_step=epoch)
                writer.add_scalar("mAP/train", mAP_value, global_step=epoch)


def validate_one_epoch(epoch, model, valid_loader, loss_func, writer, device='cpu', **kwargs):
    model.eval()
    with torch.no_grad():
        for i, d in enumerate(valid_loader):
            x = d['image']
            y = d['mask']
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss = loss_func(logits, y)
            predictions = logits_to_mask(logits)
            mAP_value = mAP(predictions, y, **kwargs)
            writer.add_scalar("Loss/valid", loss.item(), global_step=epoch)
            writer.add_scalar("mAP/valid", mAP_value, global_step=epoch)


## Architecture

Here is the UNet architecture implementation.

See the UNet paper for more information: https://arxiv.org/pdf/1505.04597v1.pdf

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        '''
        Note: We pad the convolutions here since we know the input image size is always 512x512.
        Otherwise we would do the "Overlap-tile strategy" presented in the u-net paper.
        '''
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, xx):
        return self.model(xx)


class UNet2d(nn.Module):
    ''' Paper: https://arxiv.org/pdf/1505.04597v1.pdf '''
    def __init__(self):
        super().__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

        # Down
        self.conv1 = ConvBlock(3, 64)
        self.conv2 = ConvBlock(64, 128)
        self.conv3 = ConvBlock(128, 256)
        self.conv4 = ConvBlock(256, 512)
        self.conv5 = ConvBlock(512, 1024)

        # Up
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv6 = ConvBlock(1024, 512)
        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv7 = ConvBlock(512, 256)
        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv8 = ConvBlock(256, 128)
        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv9 = ConvBlock(128, 64)
        self.conv_out = nn.Conv2d(64, 3, kernel_size=1)

    def forward(self, xx):
        # Down
        x1 = self.conv1(xx)
        xx = self.max_pool(x1)
        x2 = self.conv2(xx)
        xx = self.max_pool(x2)
        x3 = self.conv3(xx)
        xx = self.max_pool(x3)
        x4 = self.conv4(xx)
        xx = self.max_pool(x4)
        xx = self.conv5(xx)

        # Up
        xx = self.upconv1(xx)
        xx = torch.cat((x4, xx), dim=1)
        xx = self.conv6(xx)
        xx = self.upconv2(xx)
        xx = torch.cat((x3, xx), dim=1)
        xx = self.conv7(xx)
        xx = self.upconv3(xx)
        xx = torch.cat((x2, xx), dim=1)
        xx = self.conv8(xx)
        xx = self.upconv4(xx)
        xx = torch.cat((x1, xx), dim=1)
        xx = self.conv9(xx)
        return self.conv_out(xx)

## Dataset

Here is the dataset that loads the images, annotations, and peforms data augmentation.

We allow for the prediction of the background (label 0), glomerulus (label 1), and blood vessels (label 2).

In [ ]:

import torch
from tqdm import tqdm

import matplotlib.pyplot as plt


class HuBMAP(Dataset):
    ''' Training dataset for the HuBMAP Kaggle Competition '''
    def __init__(self, data_dir=os.path.join('.', 'data'), include_unsure=False):
        self.data_dir = data_dir
        # Load in the training labels
        with open(os.path.join(data_dir, 'polygons.jsonl'), 'r') as polygons_file:
            polygons = list(polygons_file)
        self.polygons = [json.loads(p) for p in polygons]

        # Load all of the training images and annotations into memory
        self.img_size = 512
        self.images = []
        self.masks = [] # target structure
        print("Loading in images and converting annotations to polygon masks...")
        for poly in tqdm(self.polygons):
            id = poly['id']
            # Get image using id
            image = Image.open(os.path.join(data_dir, 'train', f'{id}.tif'))
            self.images.append(image)

            # Get all of the different annotations for this image
            ## A single image can have multiple annotations for each type
            blood_vessel_coords = []
            glomerulus_coords = []
            unsure_coords = []
            ## Load in the type and coordinates for each annotation
            annotations = poly['annotations']
            for ann in annotations:
                type = ann['type']
                assert len(ann['coordinates']) <= 1
                coordinates = ann['coordinates'][0]
                row_indices = [c[0] for c in coordinates] 
                col_indices = [c[1] for c in coordinates]
                row_indices, col_indices = polygon(row_indices, col_indices, (self.img_size, self.img_size))
                coordinates = list(zip(row_indices, col_indices))
                if type == 'blood_vessel':
                    blood_vessel_coords.append(coordinates)
                elif type == 'glomerulus':
                    glomerulus_coords.append(coordinates)
                else:
                    unsure_coords.append(coordinates) 
            mask = self.coordinates_to_mask([item for sublist in glomerulus_coords for item in sublist])
            if include_unsure and unsure_coords is not None and len(unsure_coords) > 0:
                uns_coords = torch.tensor([item for sublist in unsure_coords for item in sublist])
                mask[uns_coords[:, 1], uns_coords[:, 0]] = 2
            bv_coords = torch.tensor([item for sublist in blood_vessel_coords for item in sublist])
            # coordinates are (x, y) like a grid
            mask[bv_coords[:, 1], bv_coords[:, 0]] = 2
            self.masks.append(mask)
        print("Done.")
        # Set up image transformations
        self.transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((512, 512), antialias=True)
        ])

    def coordinates_to_mask(self, coordinates):
        if coordinates is None or coordinates == []:
            return torch.zeros((self.img_size, self.img_size), dtype=torch.bool)
        coords = torch.tensor(coordinates)
        mask = torch.zeros((self.img_size, self.img_size), dtype=torch.bool)
        # coordinates are (x, y) like a grid
        mask[coords[:, 1], coords[:, 0]] = True
        return mask.type(torch.long)

    def __getitem__(self, i):
        return {
            'image': self.transforms(self.images[i]),
            'mask': self.masks[i],
        }

    def __len__(self):
        return len(self.images)

In [ ]:
model = UNet2d()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
memory_usage_stats(model, optimizer, 3, 'cuda')

In [ ]:
memory_usage_stats_grad_scaler(model, optimizer, 4, 'cuda')

In [ ]:
# PARAMETERS
BATCH_SIZE = 16
LR = 1e-4
WD = 0.0
MOMENTUM = 0.0
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
VALID_STEP = 5
RNG = 32
EPOCHS = 50
INCLUDE_UNSURE = True

torch.manual_seed(RNG)

writer = SummaryWriter(max_queue=1000, flush_secs=300)
dataset = HuBMAP(include_unsure=INCLUDE_UNSURE)
generator = torch.Generator().manual_seed(RNG)
train_data, valid_data = random_split(dataset, [0.9, 0.1], generator=generator)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
valid_loader = DataLoader(valid_data, batch_size=1, shuffle=False, pin_memory=False)
model = UNet2d().to(DEVICE)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

for e in tqdm(range(EPOCHS)):
    if e % VALID_STEP == 0:
        train_one_epoch(e, model, train_loader, loss_func, optimizer, writer=writer, device=DEVICE)
        validate_one_epoch(e, model, valid_loader, loss_func, writer, device=DEVICE)
    else:
        train_one_epoch(e, model, train_loader, loss_func, optimizer, device=DEVICE)
    writer.add_scalar('gpu_memory_usage', torch.cuda.memory_allocated(DEVICE), global_step=e)
writer.close()